In [1]:
import numpy as np
import pandas as pd

In [2]:
np.random.seed(42)

n = 200

# lokasi & waktu
x = np.random.uniform(0, 10, n)
y = np.random.uniform(0, 10, n)
t = np.random.uniform(0, 5, n)

# kovariat
X1 = np.random.normal(size=n)
X2 = np.random.normal(size=n)

# true coefficient surfaces
beta0 = 2
beta1 = np.sin(x / 2)
beta2 = np.cos(y / 2) * np.exp(-t / 5)

# response
Y = beta0 + beta1 * X1 + beta2 * X2 + np.random.normal(scale=0.3, size=n)

df = pd.DataFrame({
    "x": x, "y": y, "t": t,
    "X1": X1, "X2": X2,
    "Y": Y
})
df.head()

,x,y,t,X1,X2,Y
0,3.745401,6.420316,0.515619,-0.530258,2.573360,-0.395879
1,9.507143,0.841400,4.512765,-0.792873,0.059218,2.642898
2,7.319939,1.616287,2.526262,-0.107030,0.013929,1.809130
3,5.986585,8.985542,4.132287,-1.035242,-0.024125,1.990760
4,1.560186,6.064291,1.600248,-0.553649,0.198085,1.301955


In [3]:
X_ols = np.column_stack([
    np.ones(n),
    df["X1"].values,
    df["X2"].values
])

Y_vec = df["Y"].values

beta_ols = np.linalg.inv(X_ols.T @ X_ols) @ X_ols.T @ Y_vec
beta_ols

array([ 2.10274754,  0.17694018, -0.12085966])

In [4]:
def pairwise_features(df):
    n = len(df)
    feats = []

    for i in range(n):
        for j in range(n):
            feats.append([
                abs(df.x[i] - df.x[j]),
                abs(df.y[i] - df.y[j]),
                abs(df.t[i] - df.t[j])
            ])

    return np.array(feats)

Z = pairwise_features(df)
Z

array([[0.        , 0.        , 0.        ],
       [5.76174188, 5.57891681, 3.99714519],
       [3.57453823, 4.80402932, 2.01064252],
       ...,
       [1.17234714, 3.25548847, 0.65725627],
       [1.07210878, 0.03671806, 0.75710201],
       [0.        , 0.        , 0.        ]])

In [5]:
h = 6   # hidden units
lr = 0.01
epochs = 200

W1 = np.random.randn(3, h)
b1 = np.zeros(h)
W2 = np.random.randn(h, 1)
b2 = np.zeros(1)

In [6]:
def relu(z):
    return np.maximum(0, z)

def softmax_rowwise(W):
    W = W - W.max(axis=1, keepdims=True)
    expW = np.exp(W)
    return expW / expW.sum(axis=1, keepdims=True)

def forward_nn(Z):
    H = relu(Z @ W1 + b1)
    S = H @ W2 + b2
    return S.reshape(n, n)


In [7]:
def gtwr_loss(W, X, Y, lam=1e-4):
    loss = 0
    for i in range(n):
        Wi = np.diag(W[i])
        XtWX = X.T @ Wi @ X + lam * np.eye(X.shape[1])
        XtWY = X.T @ Wi @ Y
        beta_i = np.linalg.solve(XtWX, XtWY)
        yhat = X[i] @ beta_i
        loss += (Y[i] - yhat)**2
    return loss / n

In [8]:
X_loc = np.column_stack([
    np.ones(n),
    df["X1"].values,
    df["X2"].values
])

for ep in range(epochs):

    S = forward_nn(Z)
    W = softmax_rowwise(S)

    loss = gtwr_loss(W, X_loc, Y_vec)

    # numerical gradient (skripsi-safe, walau lambat)
    eps = 1e-4
    for param in [W1, W2]:
        grad = np.zeros_like(param)
        it = np.nditer(param, flags=["multi_index"], op_flags=["readwrite"]) # ini untuk iterasi elemen matriks
        while not it.finished:
            idx = it.multi_index
            param[idx] += eps
            S2 = forward_nn(Z)
            W2_ = softmax_rowwise(S2)
            loss2 = gtwr_loss(W2_, X_loc, Y_vec)
            grad[idx] = (loss2 - loss) / eps
            param[idx] -= eps
            it.iternext()
        param -= lr * grad

    if ep % 20 == 0:
        print(f"Epoch {ep}, Loss: {loss:.4f}")


Epoch 0, Loss: 0.9744
Epoch 20, Loss: 2.7065
Epoch 40, Loss: 2.9045
Epoch 60, Loss: 2.8394
Epoch 80, Loss: 3.1517
Epoch 100, Loss: 2.8559
Epoch 120, Loss: 3.0803
Epoch 140, Loss: 3.0684
Epoch 160, Loss: 2.6523
Epoch 180, Loss: 3.1601


In [9]:
betas_gtwr = np.zeros((n, 3))

for i in range(n):
    Wi = np.diag(W[i])
    XtWX = X_loc.T @ Wi @ X_loc + 1e-4*np.eye(3)
    XtWY = X_loc.T @ Wi @ Y_vec
    betas_gtwr[i] = np.linalg.solve(XtWX, XtWY)


In [10]:
beta_true = np.column_stack([
    np.full(n, beta0),
    beta1,
    beta2
])

mse_ols = np.mean((beta_ols - beta_true.mean(axis=0))**2)
mse_gtwr = np.mean((betas_gtwr - beta_true)**2)

mse_ols, mse_gtwr


(0.0038692454002160876, 0.9536058785113078)